In [1]:
#data preprocessing
import numpy as np
import pyodbc
import pandas as pd

#ml stuff
import sklearn
from sklearn.model_selection import train_test_split
from tensorflow import keras 
import tensorflow as tf
print('Tensorflow/Keras: %s' % keras.__version__)
from keras.models import Sequential 
from keras import Input 
from keras.layers import Dense
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.cluster import KMeans
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import itertools
import functools
import operator 
from more_itertools import flatten

#utils/visualization
import time
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import dill as pickle
from ydata_profiling import ProfileReport
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px

#config vars
pd.options.display.max_columns=50
pd.options.display.max_rows = 10000
verbose=1

Tensorflow/Keras: 2.12.0


# Connect to Database and get Data

In [2]:
def connect_to_db(connection_string,
                    database,
                    driver = 'SQL Server Native Client 11.0',
                    user = 'CCAdmin',
                    password = 'Miw6RjnTGmPHLYF9mG1o'
):
    connection = pyodbc.connect("Driver={"+driver+"};"
                        "Server="+connection_string+";"
                        "Database="+database+";"
                        "uid="+user+";pwd="+password+"")
    return connection

In [3]:
def get_table_data(table_name, connection):
    query = "SELECT * FROM {}".format(table_name)
    df = pd.read_sql_query(query, connection)

    return df

In [146]:
tenant = '000git'

In [7]:

#get all tables
#cursor = connect_to_db().cursor()
#tables_list = list(cursor.tables())

#connection_string='classconprocessingger.database.windows.net'

df_table_cclogattributes = get_table_data('CCLogAttributes', connect_to_db(connection_string= 'classconprocessingger.database.windows.net', database = 'T_'+tenant+''))
#df_table_cclogdocuments = get_table_data('CCLogDocuments', connect_to_db(database='T_0000xu'))
#df_table_cclogdocuments.to_pickle(df_table_cclogattributes)

#df_table_cclogattributes = df_table_cclogattributes.drop('LogTime', axis=1)
df_table_cclogattributes = df_table_cclogattributes.drop(['Zone','Attribute_DataType','LogTimeTicks'], axis=1)

df_table_cclogattributes = df_table_cclogattributes.replace('\n',' ', regex=True)
df_table_cclogattributes = df_table_cclogattributes.replace('\r',' ', regex=True)
df_table_cclogattributes.to_csv('data/cclogattributes_T_'+tenant+'.csv', index=False, header= True, encoding='utf-8')#iso-8859-15


C:\Users\lete\AppData\Local\Temp\ipykernel_15036\2297382259.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



# Data analysis


In [44]:
#df.head()
#ProfileReport(df)

In [5]:
dist = df['Delta'].value_counts()
trace = go.Pie(values=(np.array(dist)),labels=dist.index,  pull=[0, 0])
layout = go.Layout(title='Delta Distribution')
data = [trace]
fig = go.Figure(trace,layout)
fig.update_traces(marker=dict(line=dict(color='#000000', width=0.5)), textinfo='value+percent', insidetextorientation='auto')
#sfig.show()

In [6]:
dist = df['Type'].value_counts()
trace = go.Pie(values=(np.array(dist)),labels=dist.index,  pull=[0, 0.4,0.2,0.1])
layout = go.Layout(title='Type Distribution')
data = [trace]
fig = go.Figure(trace,layout)
fig.update_traces(marker=dict(line=dict(color='#000000', width=0.5)), textinfo='value+percent', insidetextorientation='auto')
#sfig.show()

# Sollerkennungswerte bestimmen


In [ ]:
def get_sollwerte(key, df):
    df_ordernum = df.loc[(df['Attribute_Name'] == 'OrderNum')] #&(df['Delta'] == False)
    dist = df_ordernum['Delta'].value_counts(normalize=True)
    print('Kreditor: '+key+'')
    
    try:
        score = 1 - dist.loc[True]
    except:
        score = dist.loc[False]

    print('q(OrderNum): '+str(score)+'') 


    df_ordernum_correct = df.loc[(df['Attribute_Name'] == 'OrderNum') &(df['Delta'] == False)] 
    documentid_list = df_ordernum_correct[['DocumentID']].values.tolist()
    merged = list(itertools.chain.from_iterable(documentid_list))
    denonimator_ordernum = len(documentid_list)
    

    df_debitor_ordernum = df.loc[(df['Attribute_Name'] == 'OrderNum')|(df['Attribute_Name'] == 'DEBITOR_NUM')|(df['Attribute_Name'] == 'VENDOR_NUM')]
    #print(df_debitor_ordernum)
    #df_debitor_ordernum.to_csv('data/debitor_preprocessing.csv', index=False, header= True, encoding='iso-8859-15')



    counter_do=0
    counter_vo=0
    for entry_1 in merged:
        speicher_entry_new = df_debitor_ordernum[df_debitor_ordernum.DocumentID == ''.join(str(entry_1))]
        if ((speicher_entry_new.Attribute_Name == 'OrderNum') & (speicher_entry_new.Delta == False)).any(): 
            if ((speicher_entry_new.Attribute_Name == 'DEBITOR_NUM')&(speicher_entry_new.Delta == False)).any():
                    counter_do += 1
            if ((speicher_entry_new.Attribute_Name == 'VENDOR_NUM')&(speicher_entry_new.Delta == False)).any():
                    counter_vo += 1
    try:
        print('q(Mandant|OrderNum): '+str(counter_do/denonimator_ordernum)+'')
        print('q(Vendor|OrderNum): '+str(counter_vo/denonimator_ordernum)+'')
    except:
         print('error')

    counter_vno=0

    df_order_delta = df.loc[(df['Attribute_Name'] == 'OrderNum')&(df['Delta'] == True)]
    df_order_delta_documentid = df_order_delta[['DocumentID']]
    order_delta_list = df_order_delta_documentid.values.tolist()
    order_delta_list_merged = list(itertools.chain.from_iterable(order_delta_list))
    denominator_order_delta_num = len(order_delta_list_merged)


    df_invoicenumber_order_delta = df.loc[(df['Attribute_Name'] == 'DEBITOR_NUM')|(df['Attribute_Name'] == 'OrderNum')]
    for entry_order_delta_invoicenumber in order_delta_list_merged:
        speicher_entry_order_delta = df_invoicenumber_order_delta[df_invoicenumber_order_delta.DocumentID == ''.join(str(entry_order_delta_invoicenumber))]
        if ((speicher_entry_order_delta.Attribute_Name == 'OrderNum') & (speicher_entry_order_delta.Delta == True)).any(): 
            if ((speicher_entry_order_delta.Attribute_Name == 'DEBITOR_NUM') & (speicher_entry_order_delta.Delta == False)).any():
                    counter_vno += 1

    if(counter_vno == 0):
        print('Da keine falsche Ordernum existiert ist Mandant|!Ordernum 0')
    else:
        print('q(Mandant|!OrderNum): '+str(counter_vno/denominator_order_delta_num)+'')
        
    df_vendor_nodelta = df.loc[(df['Attribute_Name'] == 'VENDOR_NUM')&(df['Delta'] == False)]
    df_vendor_documentid = df_vendor_nodelta[['DocumentID']]
    vendor_list = df_vendor_documentid.values.tolist()
    vendor_list_merged = list(itertools.chain.from_iterable(vendor_list))
    denominator_vendor_num = len(vendor_list)

    df_invoicenumber_vendor = df.loc[(df['Attribute_Name'] == 'InvoiceNumber')|(df['Attribute_Name'] == 'VENDOR_NUM')|(df['Attribute_Name'] == 'InvoiceDate')|(df['Attribute_Name'] == 'InvoiceDate')|(df['Attribute_Name'] == 'GrossAmount')]

    counter_inv=0
    counter_idv=0
    counter_gv=0

    for entry_vendor_invoicenumber in vendor_list_merged:
        speicher_entry_vendor = df_invoicenumber_vendor[df_invoicenumber_vendor.DocumentID == ''.join(str(entry_vendor_invoicenumber))]
        if ((speicher_entry_vendor.Attribute_Name == 'VENDOR_NUM') & (speicher_entry_vendor.Delta == False)).any(): 
            if ((speicher_entry_vendor.Attribute_Name == 'InvoiceNumber') & (speicher_entry_vendor.Delta == False)).any():
                    counter_inv += 1
            if ((speicher_entry_vendor.Attribute_Name == 'InvoiceDate') & (speicher_entry_vendor.Delta == False)).any():
                    counter_idv += 1
            if ((speicher_entry_vendor.Attribute_Name == 'GrossAmount') & (speicher_entry_vendor.Delta == False)).any():
                    counter_gv += 1

    try:
        print('q(InvoiceNumber|Vendor): '+str(counter_inv/denominator_vendor_num)+'')
        print('q(InvoiceDate|Vendor): '+str(counter_idv/denominator_vendor_num)+'')
        print('q(Gross|Vendor): '+str(counter_gv/denominator_vendor_num)+'')
    except:
        print('error')
    counter_gnv=0

    df_vendor_delta = df.loc[(df['Attribute_Name'] == 'VENDOR_NUM')&(df['Delta'] == True)]
    df_vendor_delta_documentid = df_vendor_delta[['DocumentID']]
    vendor_delta_list = df_vendor_delta_documentid.values.tolist()
    vendor_delta_list_merged = list(itertools.chain.from_iterable(vendor_delta_list))
    denominator_vendor_delta_num = len(vendor_delta_list_merged)


    df_invoicenumber_vendor_delta = df.loc[(df['Attribute_Name'] == 'GrossAmount')|(df['Attribute_Name'] == 'VENDOR_NUM')]
    for entry_vendor_delta_invoicenumber in vendor_delta_list_merged:
        speicher_entry_vendor_delta = df_invoicenumber_vendor_delta[df_invoicenumber_vendor_delta.DocumentID == ''.join(str(entry_vendor_delta_invoicenumber))]
        if ((speicher_entry_vendor_delta.Attribute_Name == 'VENDOR_NUM') & (speicher_entry_vendor_delta.Delta == True)).any(): 
            if ((speicher_entry_vendor_delta.Attribute_Name == 'GrossAmount') & (speicher_entry_vendor_delta.Delta == False)).any():
                    counter_gnv += 1

    try:
        print('q(Gross|!Vendor): '+str(counter_gnv/denominator_vendor_delta_num)+'')
    except:
        print('errror')

    counter_vom=0

    df_order_mandant_delta = df.loc[(df['Attribute_Name'] == 'OrderNum')&(df['Delta'] == True)]
    df_order_delta_new = df_order_mandant_delta.loc[(df_order_mandant_delta['Attribute_Name'] == 'Mandant')&(df['Delta'] == False)]
    df_order_delta_mandant_documentid = df_order_delta_new[['DocumentID']]
    order_delta_mandant_list = df_order_delta_mandant_documentid.values.tolist()
    order_delta_mandant_list_merged = list(itertools.chain.from_iterable(order_delta_mandant_list))
    denominator_order_delta_mandant_num = len(order_delta_mandant_list_merged)


    df_invoicenumber_order_delta_mandant = df.loc[(df['Attribute_Name'] == 'VENDOR_NUM')|(df['Attribute_Name'] == 'OrderNum')|(df['Attribute_Name'] == 'DEBITOR_NUM')]
    for entry_order_delta_mandant in order_delta_mandant_list_merged:
        speicher_entry_order_delta_mandant = df_invoicenumber_order_delta_mandant[df_invoicenumber_order_delta_mandant.DocumentID == ''.join(str(entry_order_delta_mandant))]
        if (((speicher_entry_order_delta_mandant.Attribute_Name == 'OrderNum') & (speicher_entry_order_delta_mandant.Delta == True))&((speicher_entry_order_delta_mandant.Attribute_Name == 'DEBITOR_NUM') & (speicher_entry_order_delta_mandant.Delta == False))).any(): 
            if ((speicher_entry_order_delta_mandant.Attribute_Name == 'VENDOR_NUM') & (speicher_entry_order_delta_mandant.Delta == False)).any():
                    counter_vom += 1

    if((counter_vom == 0) | (denominator_order_delta_mandant_num == 0 & counter_vom == 0)):
        print('Nulldivision q(Vendor| !OrderNum & Mandant) ungültig')
    else:
        print('q(Vendor| !OrderNum & Mandant): '+str(counter_vom/denominator_order_delta_mandant_num)+'')


    counter_vnom=0

    df_order_mandant_delta = df.loc[(df['Attribute_Name'] == 'OrderNum')&(df['Delta'] == True)]
    df_order_delta_new = df_order_mandant_delta.loc[(df_order_mandant_delta['Attribute_Name'] == 'Mandant')&(df['Delta'] == True)]
    df_order_delta_mandant_documentid = df_order_delta_new[['DocumentID']]
    order_delta_mandant_list = df_order_delta_mandant_documentid.values.tolist()
    order_delta_mandant_list_merged = list(itertools.chain.from_iterable(order_delta_mandant_list))
    denominator_order_ndelta_mandant_num = len(order_delta_mandant_list_merged)


    df_invoicenumber_order_delta_mandant = df.loc[(df['Attribute_Name'] == 'VENDOR_NUM')|(df['Attribute_Name'] == 'OrderNum')|(df['Attribute_Name'] == 'DEBITOR_NUM')]
    for entry_order_delta_mandant in order_delta_mandant_list_merged:
        speicher_entry_order_delta_mandant = df_invoicenumber_order_delta_mandant[df_invoicenumber_order_delta_mandant.DocumentID == ''.join(str(entry_order_delta_mandant))]
        if (((speicher_entry_order_delta_mandant.Attribute_Name == 'OrderNum') & (speicher_entry_order_delta_mandant.Delta == True))&((speicher_entry_order_delta_mandant.Attribute_Name == 'DEBITOR_NUM') & (speicher_entry_order_delta_mandant.Delta == True))).any(): 
            if ((speicher_entry_order_delta_mandant.Attribute_Name == 'VENDOR_NUM') & (speicher_entry_order_delta_mandant.Delta == False)).any():
                    counter_vnom += 1

    if((counter_vnom == 0) | (denominator_order_ndelta_mandant_num == 0)):
        print('Nulldivision q(Vendor| !OrderNum & !Mandant) ungültig')
    else:
        print('q(Vendor| !OrderNum & !Mandant): '+str(counter_vnom/denominator_order_ndelta_mandant_num)+'')
    
    print('--------------------------------------------------------------------------------------------')


# Data analysis


In [5]:
# Nach Kreditoren sortieren
#Nach Zeit sortieren 
def split_datframe_into_whatever(splitkey, df_new):
    df_kreditoren = df_new.loc[(df_new['Attribute_Name'] == splitkey)] 

    def split_dataframe(df_debitornum, column):
        split_dfs = {}
        for value in df_debitornum[column].unique():
            split_dfs[value] = df_debitornum[df_debitornum[column] == value][['DocumentID']]   
        return split_dfs

    dc_creditor_documentid = split_dataframe(df_kreditoren, 'Attribute_After')


    def find_corresponding_values(dict_of_dataframes, column_to_match, large_dataframe):
        result_dict = {}
        for key, df in dict_of_dataframes.items():
            temp_df = large_dataframe[large_dataframe[column_to_match].isin(df[column_to_match])]
            result_dict[key] = temp_df
        return result_dict

    dc_sorted_df_by_creditor = find_corresponding_values(dc_creditor_documentid, "DocumentID", df_new)
    return dc_sorted_df_by_creditor
    #dc_sorted_df_by_creditor[""].to_csv('data/codia.csv', index=False, header= True, encoding='utf-8')


In [6]:
def filter_df_by_time(df, start_date, end_date):
    # Filtern des DataFrames nach dem Zeitfenster
    filtered_df = df[(df['LogTime'] >= start_date) & (df['LogTime'] <= end_date)]  
    df_new = filtered_df.drop(['LogTime'], axis =1)
    # Rückgabe des gefilterten DataFrames
    return df_new

In [10]:
def get_data_values(df, value1, bool1, value2): 
    df_ordernum_correct = df.loc[(df['Attribute_Name'] == value1) &(df['Delta'] == bool1)] 
    documentid_list = df_ordernum_correct[['DocumentID']].values.tolist()
    merged = list(itertools.chain.from_iterable(documentid_list))
    denonimator_ordernum = len(documentid_list)
    
    df_debitor_ordernum = df.loc[(df['Attribute_Name'] == value1)|(df['Attribute_Name'] == value2)]

    counter_do=0
    for entry_1 in merged:
        speicher_entry_new = df_debitor_ordernum[df_debitor_ordernum.DocumentID == ''.join(str(entry_1))]
        if ((speicher_entry_new.Attribute_Name == value1) & (speicher_entry_new.Delta == bool1)).any(): 
            if ((speicher_entry_new.Attribute_Name == value2)&(speicher_entry_new.Delta == False)).any():
                    counter_do += 1

    try:
        return counter_do/denonimator_ordernum
        #return ([counter_do/denonimator_ordernum,counter_do,denonimator_ordernum])
    except:
         return pd.NA

In [83]:
def get_data_values2(df, items): 
    df_ordernum_correct = df.loc[(df['Attribute_Name'] == items[0]) &(df['Delta'] == items[1])] 
    documentid_list = df_ordernum_correct[['DocumentID']].values.tolist()
    merged = list(itertools.chain.from_iterable(documentid_list))
    denonimator_ordernum = len(documentid_list)
    
    df_debitor_ordernum = df.loc[(df['Attribute_Name'] == items[0])|(df['Attribute_Name'] == items[2])]

    counter_do=0
    for entry_1 in merged:
        speicher_entry_new = df_debitor_ordernum[df_debitor_ordernum.DocumentID == ''.join(str(entry_1))]
        if ((speicher_entry_new.Attribute_Name == items[0]) & (speicher_entry_new.Delta == items[1])).any(): 
            if ((speicher_entry_new.Attribute_Name == items[2])&(speicher_entry_new.Delta == False)).any():
                    counter_do += 1

    try:
        #return counter_do/denonimator_ordernum
        return counter_do/denonimator_ordernum,denonimator_ordernum
    except:
         return pd.NA

In [12]:
def get_data_values_two_booleans(df, value1, bool1, value2, bool2, value3):    
    counter_vom=0

    df_order_mandant_delta = df.loc[(df['Attribute_Name'] == value1)&(df['Delta'] == bool1)]
    df_order_delta_new = df_order_mandant_delta.loc[(df_order_mandant_delta['Attribute_Name'] == value2)&(df['Delta'] == bool2)]
    df_order_delta_mandant_documentid = df_order_delta_new[['DocumentID']]
    order_delta_mandant_list = df_order_delta_mandant_documentid.values.tolist()
    order_delta_mandant_list_merged = list(itertools.chain.from_iterable(order_delta_mandant_list))
    denominator_order_delta_mandant_num = len(order_delta_mandant_list_merged)


    df_invoicenumber_order_delta_mandant = df.loc[(df['Attribute_Name'] == value1)|(df['Attribute_Name'] == value2)|(df['Attribute_Name'] == value3)]
    for entry_order_delta_mandant in order_delta_mandant_list_merged:
        speicher_entry_order_delta_mandant = df_invoicenumber_order_delta_mandant[df_invoicenumber_order_delta_mandant.DocumentID == ''.join(str(entry_order_delta_mandant))]
        if (((speicher_entry_order_delta_mandant.Attribute_Name == value1) & (speicher_entry_order_delta_mandant.Delta == bool1))&((speicher_entry_order_delta_mandant.Attribute_Name == value2) & (speicher_entry_order_delta_mandant.Delta == bool2))).any(): 
            if ((speicher_entry_order_delta_mandant.Attribute_Name == value3) & (speicher_entry_order_delta_mandant.Delta == False)).any():
                    counter_vom += 1

    #if((counter_vom == 0) | (denominator_order_delta_mandant_num == 0 & counter_vom == 0)):
    try:
        return counter_vom/denominator_order_delta_mandant_num
        #return ([counter_vom/denominator_order_delta_mandant_num,counter_vom,denominator_order_delta_mandant_num])
    except:
         return pd.NA

In [126]:
def get_data_values_two_booleans2(df, items):    
    counter_vom=0

    df_order_mandant_delta = df.loc[(df['Attribute_Name'] == items[0])&(df['Delta'] == items[1])]
    df_order_delta_new = df_order_mandant_delta.loc[(df_order_mandant_delta['Attribute_Name'] == items[2])&(df['Delta'] == items[3])]
    df_order_delta_mandant_documentid = df_order_delta_new[['DocumentID']]
    order_delta_mandant_list = df_order_delta_mandant_documentid.values.tolist()
    order_delta_mandant_list_merged = list(itertools.chain.from_iterable(order_delta_mandant_list))
    denominator_order_delta_mandant_num = len(order_delta_mandant_list_merged)


    df_invoicenumber_order_delta_mandant = df.loc[(df['Attribute_Name'] == items[0])|(df['Attribute_Name'] == items[2])|(df['Attribute_Name'] == items[4])]
    for entry_order_delta_mandant in order_delta_mandant_list_merged:
        speicher_entry_order_delta_mandant = df_invoicenumber_order_delta_mandant[df_invoicenumber_order_delta_mandant.DocumentID == ''.join(str(entry_order_delta_mandant))]
        if (((speicher_entry_order_delta_mandant.Attribute_Name == items[0]) & (speicher_entry_order_delta_mandant.Delta == items[1]))&((speicher_entry_order_delta_mandant.Attribute_Name == items[2]) & (speicher_entry_order_delta_mandant.Delta == items[3]))).any(): 
            if ((speicher_entry_order_delta_mandant.Attribute_Name == items[4]) & (speicher_entry_order_delta_mandant.Delta == False)).any():
                    counter_vom += 1

    #if((counter_vom == 0) | (denominator_order_delta_mandant_num == 0 & counter_vom == 0)):
    try:
        #return counter_vom/denominator_order_delta_mandant_num
        return counter_vom/denominator_order_delta_mandant_num,denominator_order_delta_mandant_num
    except:
         return pd.NA

In [135]:
def get_single_value(df, value):
    df_ordernum = df.loc[(df['Attribute_Name'] == value)] 
    dist = df_ordernum['Delta'].value_counts(normalize=True)
    try:
        score_false = dist.loc[False]
        if(score_false >= 0):
            score = score_false
        elif(score_false == 0):
            score = dist.loc[True]
    except:
        return pd.NA

        
    return score, len(df_ordernum)



In [147]:
list1 = [['OrderNum',False,'DEBITOR_NUM'],
         ['OrderNum',False,'VENDOR_NUM'],
         ['OrderNum',True,'DEBITOR_NUM'],
         ['VENDOR_NUM',False,'InvoiceNumber'],
         ['VENDOR_NUM',True,'InvoiceNumber'],
         ['VENDOR_NUM',False,'InvoiceDate'],
         ['VENDOR_NUM',False,'GrossAmount'],
         ['VENDOR_NUM',True,'GrossAmount'],
         ['VENDOR_NUM',False,'NetAmount1'],
         ['VENDOR_NUM',True,'NetAmount1'],
         ['VENDOR_NUM',False,'VatAmount1'],
         ['VENDOR_NUM',True,'VatAmount1'],
         ]

list2 = [['OrderNum',True,'DEBITOR_NUM',False,'VENDOR_NUM'],
         ['OrderNum',True,'DEBITOR_NUM',True,'VENDOR_NUM'],
         ]

df_results = pd.DataFrame(columns=['q(OrderNum)','q(DEBITOR_NUM|OrderNum)',
         'q(VENDOR_NUM|OrderNum)',
         'q(DEBITOR_NUM|!OrderNum)',
         'q(InvoiceNumber|VENDOR_NUM)',
         'q(InvoiceNumber|!VENDOR_NUM)',
         'q(InvoiceDate|VENDOR_NUM)',
         'q(GrossAmount|VENDOR_NUM)',
         'q(GrossAmount|!VENDOR_NUM)',
         'q(NetAmount1|VENDOR_NUM)',
         'q(NetAmount1|!VENDOR_NUM)',
         'q(VatAmount1|VENDOR_NUM)',
         'q(VatAmount1|!VENDOR_NUM)',
         'q(VENDOR_NUM | !OrderNum & DEBITOR_NUM)',
         'q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)',
         'q(VENDOR_NUM)',])

df = pd.read_csv('data/cclogattributes_T_'+tenant+'.csv', encoding='utf-8')

df_new = filter_df_by_time(df,'2022-12-20 09:44:23.030','2023-04-19 12:28:20.000')
dc_sorted_df = split_datframe_into_whatever('DEBITOR_NUM', df_new)


for key, df in dc_sorted_df.items():
    result_list = []
    #get_sollwerte(key, df)
    ordernum = get_single_value(df, 'OrderNum')
    #print(''+key+': '+'OrderNum = '+str(ordernum)+'')
    result_list.append(ordernum)

    for items1 in list1:
        x = get_data_values2(df, items1)
        result_list.append(x)

    for items2 in list2:
        y = get_data_values_two_booleans2(df, items2)
        result_list.append(y)
        

    vendornum = get_single_value(df, 'VENDOR_NUM')
    #print(''+key+': '+'VENDOR_NUM = '+str(vendornum)+'')
    result_list.append(vendornum)

    df_results.loc[key] = result_list
    print(result_list)

[(0.765748031496063, 508), (0.9974293059125964, 389), (0.9948586118251928, 389), (0.9495798319327731, 119), (0.9891402714932127, 1105), (0.9425287356321839, 87), (0.9493212669683257, 1105), (0.6470588235294118, 1105), (0.39080459770114945, 87), (0.4316742081447964, 1105), (0.40229885057471265, 87), (0.4579185520361991, 1105), (0.4482758620689655, 87), <NA>, <NA>, (0.927013422818792, 1192)]
[(<NA>, 0), <NA>, <NA>, <NA>, (0.7692307692307693, 273), (0.9024390243902439, 41), (0.8571428571428571, 273), (0.4175824175824176, 273), (0.5365853658536586, 41), (0.6959706959706959, 273), (0.6097560975609756, 41), (0.6959706959706959, 273), (0.7560975609756098, 41), <NA>, <NA>, (0.8694267515923567, 314)]
[(1.0, 6), (1.0, 6), (0.0, 6), <NA>, (0.954248366013072, 1071), (0.8726114649681529, 157), (0.8123249299719888, 1071), (0.7441643323996265, 1071), (0.5923566878980892, 157), (0.23996265172735762, 1071), (0.21019108280254778, 157), (0.25770308123249297, 1071), (0.20382165605095542, 157), <NA>, <NA>,

In [148]:
#df_results = df_results.sort_values(by=['q(GrossAmount|VENDOR_NUM)'],ascending=True)
import ast
#df_results_new = pd.read_csv('data/sollwerte_debitor_mit_ordernum.csv', encoding='utf-8')
#df_results_new = pd.read_clipboard(header=None, quotechar='"', sep=',', converters={1:ast.literal_eval})
#df_results.to_csv('data/sollwerte_debitor_mit_ordernum.csv', index=True, header= True, encoding='utf-8')#iso-8859-15
#df_results = df_results.reset_index(drop=True)

#df_results.drop(2, inplace=True)
#display(df_results)
display(df_results)

,q(OrderNum),q(DEBITOR_NUM|OrderNum),q(VENDOR_NUM|OrderNum),q(DEBITOR_NUM|!OrderNum),q(InvoiceNumber|VENDOR_NUM),q(InvoiceNumber|!VENDOR_NUM),q(InvoiceDate|VENDOR_NUM),q(GrossAmount|VENDOR_NUM),q(GrossAmount|!VENDOR_NUM),q(NetAmount1|VENDOR_NUM),q(NetAmount1|!VENDOR_NUM),q(VatAmount1|VENDOR_NUM),q(VatAmount1|!VENDOR_NUM),q(VENDOR_NUM | !OrderNum & DEBITOR_NUM),q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM),q(VENDOR_NUM)
1200,"(0.765748031496063, 508)","(0.9974293059125964, 389)","(0.9948586118251928, 389)","(0.9495798319327731, 119)","(0.9891402714932127, 1105)","(0.9425287356321839, 87)","(0.9493212669683257, 1105)","(0.6470588235294118, 1105)","(0.39080459770114945, 87)","(0.4316742081447964, 1105)","(0.40229885057471265, 87)","(0.4579185520361991, 1105)","(0.4482758620689655, 87)",NaN,NaN,"(0.927013422818792, 1192)"
1100,"(<NA>, 0)",NaN,NaN,NaN,"(0.7692307692307693, 273)","(0.9024390243902439, 41)","(0.8571428571428571, 273)","(0.4175824175824176, 273)","(0.5365853658536586, 41)","(0.6959706959706959, 273)","(0.6097560975609756, 41)","(0.6959706959706959, 273)","(0.7560975609756098, 41)",NaN,NaN,"(0.8694267515923567, 314)"
2900,"(1.0, 6)","(1.0, 6)","(0.0, 6)",NaN,"(0.954248366013072, 1071)","(0.8726114649681529, 157)","(0.8123249299719888, 1071)","(0.7441643323996265, 1071)","(0.5923566878980892, 157)","(0.23996265172735762, 1071)","(0.21019108280254778, 157)","(0.25770308123249297, 1071)","(0.20382165605095542, 157)",NaN,NaN,"(0.8721498371335505, 1228)"
1400,"(0.7306122448979592, 245)","(1.0, 179)","(1.0, 179)","(1.0, 66)","(0.9812206572769953, 426)","(0.9701492537313433, 134)","(0.9765258215962441, 426)","(0.7793427230046949, 426)","(0.5, 134)","(0.5727699530516432, 426)","(0.5597014925373134, 134)","(0.6455399061032864, 426)","(0.5746268656716418, 134)",NaN,NaN,"(0.7607142857142857, 560)"
1300,"(0.3333333333333333, 9)","(1.0, 3)","(0.0, 3)","(1.0, 6)","(0.9770408163265306, 392)","(0.9459459459459459, 37)","(0.7244897959183674, 392)","(0.4770408163265306, 392)","(0.5405405405405406, 37)","(0.41836734693877553, 392)","(0.35135135135135137, 37)","(0.6045918367346939, 392)","(0.4594594594594595, 37)",NaN,NaN,"(0.9137529137529138, 429)"
3300,"(<NA>, 0)",NaN,NaN,NaN,"(0.9597701149425287, 174)","(0.7837837837837838, 37)","(0.7471264367816092, 174)","(0.8563218390804598, 174)","(0.8108108108108109, 37)","(0.7701149425287356, 174)","(0.2972972972972973, 37)","(0.8103448275862069, 174)","(0.40540540540540543, 37)",NaN,NaN,"(0.8246445497630331, 211)"
3500,"(<NA>, 0)",NaN,NaN,NaN,"(0.8975903614457831, 166)","(0.8636363636363636, 22)","(0.7409638554216867, 166)","(0.8734939759036144, 166)","(0.5, 22)","(0.8674698795180723, 166)","(0.5, 22)","(0.9578313253012049, 166)","(0.8636363636363636, 22)",NaN,NaN,"(0.8829787234042553, 188)"


In [139]:
def sollwerte_anhand_wahrscheinlichkeit(df):
    sollwerte = pd.Series()
    for column in df.columns:
        #df.values.tolist()
        probability = []
        count = []
        for entry in df[column]:
            #df.values.tolist()
            if (pd.isnull(entry)):
                continue      
            else:
                probability.append(entry[0])
                count.append(entry[1])

        wholecount = sum(count)
        gesamtwahrscheinlichkeit = sum([p * h / wholecount for p, h in zip(probability, count)])  #p = probability * count/wholcount

        sollwerte[column] = gesamtwahrscheinlichkeit
    return sollwerte

sollwerte = sollwerte_anhand_wahrscheinlichkeit(df_results)
#sollwerte.to_csv('data/sollwerte.csv', index=True, header= True, encoding='utf-8')
print(sollwerte)

q(OrderNum)                                      NaN
q(DEBITOR_NUM|OrderNum)                     0.000000
q(VENDOR_NUM|OrderNum)                      0.000000
q(DEBITOR_NUM|!OrderNum)                    0.000000
q(InvoiceNumber|VENDOR_NUM)                 0.991736
q(InvoiceNumber|!VENDOR_NUM)                0.800000
q(InvoiceDate|VENDOR_NUM)                   1.000000
q(GrossAmount|VENDOR_NUM)                   0.983471
q(GrossAmount|!VENDOR_NUM)                  0.800000
q(NetAmount1|VENDOR_NUM)                    0.867769
q(NetAmount1|!VENDOR_NUM)                   0.800000
q(VatAmount1|VENDOR_NUM)                    0.884298
q(VatAmount1|!VENDOR_NUM)                   1.000000
q(VENDOR_NUM | !OrderNum & DEBITOR_NUM)     0.000000
q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)    0.000000
q(VENDOR_NUM)                               0.960317
dtype: float64


C:\Users\lete\AppData\Local\Temp\ipykernel_19916\2528527961.py:2: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [132]:
abc = pd.read_csv('data/sollwerte.csv', encoding='utf-8')
display(abc)

,Unnamed: 0,0
0,q(OrderNum),1.000
1,q(DEBITOR_NUM|OrderNum),1.000
2,q(VENDOR_NUM|OrderNum),0.797
3,q(DEBITOR_NUM|!OrderNum),0.734
4,q(InvoiceNumber|VENDOR_NUM),0.963
5,q(InvoiceNumber|!VENDOR_NUM),0.906
6,q(InvoiceDate|VENDOR_NUM),0.974
7,q(GrossAmount|VENDOR_NUM),0.981
8,q(GrossAmount|!VENDOR_NUM),0.800
9,q(NetAmount1|VENDOR_NUM),0.878


In [145]:
print(df_results)

    q(OrderNum) q(DEBITOR_NUM|OrderNum) q(VENDOR_NUM|OrderNum)  \
JRG   (<NA>, 0)                     NaN                    NaN   
01    (<NA>, 0)                     NaN                    NaN   

    q(DEBITOR_NUM|!OrderNum) q(InvoiceNumber|VENDOR_NUM)  \
JRG                      NaN   (0.9916666666666667, 120)   
01                       NaN                    (1.0, 1)   

    q(InvoiceNumber|!VENDOR_NUM) q(InvoiceDate|VENDOR_NUM)  \
JRG                     (0.8, 5)                (1.0, 120)   
01                           NaN                  (1.0, 1)   

     q(GrossAmount|VENDOR_NUM) q(GrossAmount|!VENDOR_NUM)  \
JRG  (0.9833333333333333, 120)                   (0.8, 5)   
01                    (1.0, 1)                        NaN   

      q(NetAmount1|VENDOR_NUM) q(NetAmount1|!VENDOR_NUM)  \
JRG  (0.8666666666666667, 120)                  (0.8, 5)   
01                    (1.0, 1)                       NaN   

      q(VatAmount1|VENDOR_NUM) q(VatAmount1|!VENDOR_NUM)  \
JRG  (0.

In [144]:
medians = df_results.median()
print(medians)

for col in df_results.columns:
    print(col)
    # Berechne den Median dieser Spalte
    median = medians[col]
    print(median)
    # Berechne die Standardabweichung dieser Spalte
    std = df_results[col].std()
    # 2-fache der Standardabweichung vom Median abweichen
    outliers = df_results[(df_results[col] - median).abs() > 2 * std]

    if not outliers.empty:
        print(f"Ausreißer in Spalte {col}:")
        #print(outliers)
        print(outliers.index)

q(DEBITOR_NUM|OrderNum)                    NaN
q(VENDOR_NUM|OrderNum)                     NaN
q(DEBITOR_NUM|!OrderNum)                   NaN
q(VENDOR_NUM | !OrderNum & DEBITOR_NUM)    NaN
q(VENDOR_NUM| !OrderNum & ! DEBITOR_NUM)   NaN
dtype: float64
q(OrderNum)


C:\Users\lete\AppData\Local\Temp\ipykernel_19916\3862545096.py:1: FutureWarning:

The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.



KeyError: 'q(OrderNum)'

In [117]:
wahrscheinlichkeiten = [0.2, 0.3, 0.5]
häufigkeiten = [10, 20, 30]

# Berechnen der Gesamthäufigkeit
gesamthäufigkeit = sum(häufigkeiten)

# Berechnen der Gesamtwahrscheinlichkeit
gesamtwahrscheinlichkeit = sum([p * h / gesamthäufigkeit for p, h in zip(wahrscheinlichkeiten, häufigkeiten)])

# Ausgabe des Ergebnisses
print(gesamtwahrscheinlichkeit)

0.3833333333333333


# Machine Learning


## Data preprocessing

In [54]:
df = pd.read_csv('data/cclogattributes_T0012hb.csv', encoding='iso-8859-15')

vergleiche_spalten = lambda x: 'master_data_error' if x['Attribute_Name'] == 'VENDOR_NUM' and x['Attribute_Before'] != x['Attribute_After'] else'good' if x['Delta'] == False and x['Type'] == 0 else 'undefined'
#else 'master_data_error' if x['Attribute_Name'] == 'VENDOR_NAME' and x['Attribute_Before'] != x['Attribute_After'] \
#else 'master_data_error' if x['Attribute_Name'] == 'VENDOR_STR' and x['Attribute_Before'] != x['Attribute_After'] \
#else 'master_data_error' if x['Attribute_Name'] == 'VENDOR_CITY' and x['Attribute_Before'] != x['Attribute_After'] \
#else 'master_data_error' if x['Attribute_Name'] == 'VENDOR_ZIP_CODE' and x['Attribute_Before'] != x['Attribute_After'] \

df['Suggestion'] = df.apply(vergleiche_spalten, axis=1)

df = df.drop('LogTimeTicks', axis=1)
#df = df.drop('DocumentID', axis=1)

df.to_csv('data/after_preprocessing.csv', index=False, header= True, encoding='iso-8859-15')

X = df.loc[ : , df.columns != 'Suggestion']
Y = df.loc[ : , df.columns == 'Suggestion']


In [ ]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

X = handle_non_numerical_data(X)


def Encoder(df):
          columnsToEncode = list(df.select_dtypes(include=['category','object','bool']))
          le = LabelEncoder()
          for feature in columnsToEncode:
              try:
                  df[feature] = le.fit_transform(df[feature])
              except:
                  print('Error encoding '+feature)
          return df


Y = Encoder(Y)
print(Y)
pd.DataFrame(Y).to_csv("data/after_numerical.csv")


print(X)
X.shape[0]
Y.shape[0]

In [4]:
#Create training and testing samples
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=None, shuffle=False)

pd.DataFrame(y_train).to_csv("data/y_train.csv")

## Create Model and Train

In [5]:
def create_model(optimizer='adam', init='uniform'):
    # create model
    if verbose: print("**Create model with optimizer: %s; init: %s" % (optimizer, init) )
    model = Sequential()
    model.add(Dense(16, input_dim=X_train.shape[1], kernel_initializer=init, activation='relu')) #
    model.add(Dense(512, kernel_initializer=init, activation='relu'))
    model.add(Dense(256, kernel_initializer=init, activation='relu'))
    model.add(Dense(3, kernel_initializer=init, activation='softmax'))
    # Compile model
    model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adadelta(), metrics=['acc']) #keras.optimizers.Adadelta() and sparse_categorical_accuracy
    return model

In [46]:
run_gridsearch = False

if run_gridsearch:
    
    start_time = time.time()
    if verbose: print (time.strftime( "%H:%M:%S " + "GridSearch started ... " ) )
    optimizers = ['rmsprop'] # , 'adam'
    inits = ['glorot_uniform'] #, 'normal', 'uniform'
    epochs = [5, 50, 100, 300]
    batches = [5, 32, 64]
    
    model = KerasClassifier(build_fn=create_model, verbose=verbose)
    
    param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
    grid = GridSearchCV(estimator=model, param_grid=param_grid)
    grid_result = grid.fit(X_train, y_train)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    if verbose: 
        for mean, stdev, param in zip(means, stds, params):
            print("%f (%f) with: %r" % (mean, stdev, param))
        elapsed_time = time.time() - start_time  
        print ("Time elapsed: ",timedelta(seconds=elapsed_time))
        
    best_epochs = grid_result.best_params_['epochs']
    best_batch_size = grid_result.best_params_['batch_size']
    best_init = grid_result.best_params_['init']
    best_optimizer = grid_result.best_params_['optimizer']
    
else:
    # best paramters
    best_epochs = 300
    best_batch_size = 64
    best_init = 'glorot_uniform'
    best_optimizer = 'rmsprop'

In [ ]:
results_df = pd.DataFrame(grid_result.cv_results_)
pd.DataFrame(results_df).to_csv('data/grid_search_results.csv')
results_df = results_df.sort_values(by=["rank_test_score"])
results_df = results_df.set_index(
    results_df["params"].apply(lambda x: "_".join(str(val) for val in x.values()))
).rename_axis("kernel")
results_df[["params", "rank_test_score", "mean_test_score", "std_test_score"]]

In [ ]:
model_scores = results_df.filter(regex=r"split\d*_test_score")

# plot 30 examples of dependency between cv fold and AUC scores
fig, ax = plt.subplots()
sns.lineplot(
    data=model_scores.transpose().iloc[:30],
    dashes=False,
    palette="Set1",
    marker="o",
    alpha=0.5,
    ax=ax,
)
ax.set_xlabel("CV test fold", size=12, labelpad=10)
ax.set_ylabel("Model AUC", size=12)
ax.tick_params(bottom=True, labelbottom=False)
ax.legend(loc=4, prop={'size': 6})

plt.show()


In [ ]:
model_pred = KerasClassifier(build_fn=create_model, optimizer=best_optimizer, init=best_init, epochs=best_epochs, batch_size=best_batch_size, verbose=verbose)
model_pred.fit(X_train, y_train)

prediction = model_pred.predict(X_test)

# Evaluation

In [ ]:
print(classification_report(y_test, prediction, zero_division=False))
accuracy_score(y_test, prediction)

In [ ]:
print(prediction)
pd.DataFrame(prediction).to_csv('data/prediction.csv')

In [50]:
pickle.dump(model_pred, open('models/classification_model.h5', 'wb'))

In [ ]:
with open('models/classification_model.h5', 'rb') as pickle_file:
    content = pickle.load(pickle_file)

prediction = content.predict(X_test)
print(classification_report(y_test, prediction, zero_division=False))


In [ ]:
import pyautogui
import time
while True:
    for i in range(0,100):
        pyautogui.moveTo(10,10)
        pyautogui.leftClick()
        pyautogui.moveTo(1000,1000)
        pyautogui.leftClick()

        time.sleep(2)